一.VGG16复现

VGG网络结构特点

卷积层设计：

统一卷积核尺寸：VGG 模型大量使用 3×3 的小卷积核，这是它的一大标志性特点。相比于大尺寸卷积核（如 5×5、7×7 ），3×3 卷积核有两个主要优势。一方面，多个 3×3 卷积核堆叠的效果可以近似替代大卷积核，且参数量更少。比如，两个 3×3 卷积核堆叠相当于一个 5×5 卷积核的感受野，但参数量从 5×5× 输入通道数 × 输出通道数，减少到了（3×3× 输入通道数 × 中间通道数）+（3×3× 中间通道数 × 输出通道数）；另一方面，3×3 卷积核能更好地保留图像的细节信息，且增加了网络的非线性表达能力，因为每一个卷积层后都可以添加激活函数 。

固定步长和填充：卷积层通常使用步长为 1，填充为 1 的设置，这样可以保证在进行卷积操作后，特征图的尺寸保持不变（计算公式：输出尺寸 = (输入尺寸 - 卷积核尺寸 + 2× 填充) / 步长 + 1，当卷积核为 3×3、步长为 1、填充为 1 时，输出尺寸等于输入尺寸 ）。

池化层设计：在连续的几个卷积层之后，会跟一个最大池化层，池化核大小为 2×2，步长为 2。通过池化操作，特征图的尺寸会减半，这样逐步降低特征图的空间维度，同时增加特征图的通道数，使网络能够学习到更高级的语义信息。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/7.png?versionId=CAEQFRiBgMD2zKyfxxciIGZhMDQ0Y2UyYTA5ZjQ1NjhhMWNjNDQ1Njg3YTFiODZh)

In [31]:
import torch
from torch import nn
from torch.nn import functional as F
from torchinfo import summary

In [32]:
class VGG16(nn.Module):
    def __init__(self):
        super().__init__()
        self.features_ = nn.Sequential(nn.Conv2d(3,64,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(64,64,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(64,128,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(128,128,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(128,256,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(256,256,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(256,256,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(256,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                      )
        self.clf_ = nn.Sequential(nn.Dropout(0.5)
                                  ,nn.Linear(512*7*7,4096),nn.ReLU(inplace=True)
                                  ,nn.Dropout(0.5)
                                  ,nn.Linear(4096,4096),nn.ReLU(inplace=True)
                                  ,nn.Linear(4096,1000),nn.Softmax(dim=1)
                                 )
    
    # [10, 512, 7, 7]
    def forward(self,x):
        x = self.features_(x) #用特征提取的架构提取特征
        x = x.view(-1,512*7*7) #调整数据结构，拉平数据
        output = self.clf_(x)
        return output

In [33]:
net=VGG16()
summary(net,input_size=(10,3,224,224),device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
VGG16                                    [10, 1000]                --
├─Sequential: 1-1                        [10, 512, 7, 7]           --
│    └─Conv2d: 2-1                       [10, 64, 224, 224]        1,792
│    └─ReLU: 2-2                         [10, 64, 224, 224]        --
│    └─Conv2d: 2-3                       [10, 64, 224, 224]        36,928
│    └─ReLU: 2-4                         [10, 64, 224, 224]        --
│    └─MaxPool2d: 2-5                    [10, 64, 112, 112]        --
│    └─Conv2d: 2-6                       [10, 128, 112, 112]       73,856
│    └─ReLU: 2-7                         [10, 128, 112, 112]       --
│    └─Conv2d: 2-8                       [10, 128, 112, 112]       147,584
│    └─ReLU: 2-9                         [10, 128, 112, 112]       --
│    └─MaxPool2d: 2-10                   [10, 128, 56, 56]         --
│    └─Conv2d: 2-11                      [10, 256, 56, 56]         29

用于确定输入线性层之前的特征数据

In [34]:
data=torch.ones(size=(10,3,224,224))
net = nn.Sequential(nn.Conv2d(3,64,3,padding=1),
                    nn.ReLU(inplace=True)
                                       ,nn.Conv2d(64,64,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(64,128,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(128,128,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(128,256,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(256,256,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(256,256,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(256,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                      )
net(data).shape

torch.Size([10, 512, 7, 7])